Importing pandas and numpy

In [53]:
import pandas as pd
import numpy as np

Loading training set into dataframe called df

In [54]:
df = pd.read_csv('../data/raw/train.csv',low_memory=False)

In [55]:
df['ht'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 56091 entries, 0 to 56090
Series name: ht
Non-Null Count  Dtype 
--------------  ----- 
56011 non-null  object
dtypes: object(1)
memory usage: 438.3+ KB


converting height to cm

In [56]:
# function to convert 'ht' column to height in centimeters
def feet_inches_to_cm(height):
    if isinstance(height, str):  # Check if the value is a string
        parts = height.split('-')
        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            feet = int(parts[0])
            inches = int(parts[1])
        else:
            try:
                feet = int(parts[0])
                inches = 0
            except ValueError:
                return None  # Handle unexpected or missing values
        total_inches = feet * 12 + inches
        cm = total_inches * 2.54
        return float(cm)  # Convert calculated height to float
    return None  

In [57]:
# applying the conversion function to the 'ht' column
df['ht_cm'] = df['ht'].apply(feet_inches_to_cm)

In [58]:
df['ht_cm'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 56091 entries, 0 to 56090
Series name: ht_cm
Non-Null Count  Dtype  
--------------  -----  
52090 non-null  float64
dtypes: float64(1)
memory usage: 438.3 KB


Removing nulls

In [51]:
df=df.dropna()

Splitting data into features and target

In [59]:
features = df.drop(['player_id','drafted','ht'],axis=1)
target = df['drafted']

One-hot-encoding

In [60]:
features = pd.get_dummies(features)

In [61]:
print(features.shape,target.shape)

(56091, 504) (56091,)


In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, random_state=42)

In [64]:
from sklearn.preprocessing import StandardScaler

In [65]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

Training the model

In [66]:
from xgboost import XGBClassifier

In [67]:
# Model training (XGBoost)
xgb = XGBClassifier()
xgb.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [68]:
from sklearn.metrics import roc_auc_score

In [69]:
# Model evaluation (Random Forest)
y_pred_prob = xgb.predict_proba(X_val_scaled)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_prob)
print(f'XGBoost AUROC: {roc_auc:.4f}')

XGBoost AUROC: 0.9988


In [70]:
test_data = pd.read_csv('../data/raw/test.csv',low_memory=False)

In [71]:
test_data_processed = pd.get_dummies(test_data)
test_data_processed = test_data_processed.reindex(columns=features.columns, fill_value=0)  # Align columns
test_data_scaled = scaler.transform(test_data_processed)
predictions = xgb.predict_proba(test_data_scaled)[:,1]

In [72]:
test_data['drafted'] = predictions
print(test_data[['player_id', 'drafted']])

                                 player_id       drafted
0     cf302b4d-84f7-4124-a25d-a75eed31978b  2.229679e-06
1     f91837cd-4f49-4b70-963d-aeb82c6ce3da  2.794986e-06
2     53ec2a29-1e7d-4c6d-86d7-d60d02af8916  1.329532e-05
3     32402798-471c-4a54-8cb4-29cd95199014  3.825915e-06
4     73b960f9-27b8-4431-9d23-a760e9bbc360  2.705162e-06
...                                    ...           ...
4965  a25ee55f-02a3-4f8e-8194-a5f427e14e7c  3.745262e-06
4966  d0d9f45e-7b01-44b3-8d40-514ec338611d  9.100883e-07
4967  f8df22c4-1602-4fab-896d-8820951aae2f  6.377062e-07
4968  b791c69a-f769-4163-afda-051a6fd20a9d  6.377062e-07
4969  18b51f5d-4746-4121-88fd-c8d0a1399130  1.612597e-06

[4970 rows x 2 columns]


In [73]:
test_data[['player_id', 'drafted']].to_csv('predictions_xgb_refined.csv',index=False)

In [14]:
from catboost import CatBoostClassifier

Training the model

In [15]:
# Model training (CatBoost)
cb = CatBoostClassifier(n_estimators=150, random_state=8, verbose=0)
cb.fit(X_train_scaled, y_train)

In [16]:
from sklearn.metrics import roc_auc_score

In [17]:
# Model evaluation (LightGBM)
y_pred_prob = cb.predict_proba(X_val_scaled)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_prob)
print(f'Cat Boost AUROC: {roc_auc:.4f}')

Cat Boost AUROC: 0.8991


Final testing

In [18]:
test_data = pd.read_csv('../data/raw/test.csv',low_memory=False)

In [19]:
test_data_processed = pd.get_dummies(test_data)
test_data_processed = test_data_processed.reindex(columns=features.columns, fill_value=0) 
test_data_scaled = scaler.transform(test_data_processed)
predictions = cb.predict_proba(test_data_scaled)[:,1]

In [20]:
test_data['drafted'] = predictions
print(test_data[['player_id', 'drafted']])

                                 player_id   drafted
0     cf302b4d-84f7-4124-a25d-a75eed31978b  0.156081
1     f91837cd-4f49-4b70-963d-aeb82c6ce3da  0.135288
2     53ec2a29-1e7d-4c6d-86d7-d60d02af8916  0.363793
3     32402798-471c-4a54-8cb4-29cd95199014  0.231430
4     73b960f9-27b8-4431-9d23-a760e9bbc360  0.603604
...                                    ...       ...
4965  a25ee55f-02a3-4f8e-8194-a5f427e14e7c  0.079086
4966  d0d9f45e-7b01-44b3-8d40-514ec338611d  0.012200
4967  f8df22c4-1602-4fab-896d-8820951aae2f  0.014382
4968  b791c69a-f769-4163-afda-051a6fd20a9d  0.014207
4969  18b51f5d-4746-4121-88fd-c8d0a1399130  0.064051

[4970 rows x 2 columns]


In [21]:
print(test_data.shape)

(4970, 64)


Saving to csv

In [22]:
test_data[['player_id', 'drafted']].to_csv('predictions_cat.csv',index=False)